In [1]:
import requests
import datetime
import math

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
# Clean values inside the td tag
def clean(v):
    return v.replace('\t', '').replace('\r', '').replace('\n', ' ').replace(u'\xa0', '').replace('--', '')

In [3]:
# Check if value is nan or empty string
def is_empty(v):
    return (type(v) == float and math.isnan(v)) or (type(v) == str and len(v.strip()) == 0)

In [4]:
# Request the html page and webscrap for data
# Returns a data frame
def get_data(date, station):
    
    station_name = station.split('/')[1]
    
    date_str = date.strftime('%d/%m/%Y')
    
    print(station_name, date_str)

    form_input = {'texData': f'{date_str}', 'selEst': f'{station}'}

    page = requests.post(URL, data = form_input)

    soup = BeautifulSoup(page.content, 'html.parser')
    
    parsed_data = {'Date': date_str, 'Station': station_name}

    for table in soup.find_all('table', class_='font01')[1:]:
        data = []
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            data.append(clean(tds[len(tds) - 1].text))

        parsed_data[(data[0] + ' ' + data[1]).strip()] = data[2:]
    
    df = pd.DataFrame(parsed_data)
    
    # Filter empty rows
    df = df[df.applymap(is_empty).sum(axis=1) < (df.shape[1] - 3)]
    
    return df

In [5]:
# Air quality url
URL = 'https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php'

In [6]:
STATIONS = [
    '65/Americana',
    '72/Araçatuba',
    '71/Araraquara',
    '73/Bauru',
    '42/Campinas-Centro',
    '41/Campinas-Taquaral',
    '43/Campinas-V.União',
    '37/Capão Redondo',
    '28/Carapicuíba',
    '81/Catanduva',
    '10/Cerqueira César',
    '31/Cid.Universitária-USP-Ipen',
    '8/Congonhas',
    '24/Cubatão-Centro',
    '25/Cubatão-V.Parisi',
    '30/Cubatão-Vale do Mogi',
    '15/Diadema',
    '29/Grajaú-Parelheiros',
    '62/Guaratinguetá',
    '35/Guarulhos-Paço Municipal',
    '40/Guarulhos-Pimentas',
    '5/Ibirapuera',
    '34/Interlagos',
    '33/Itaim Paulista',
    '50/Itaquera',
    '54/Jacareí',
    '75/Jaú',
    '74/Jundiaí',
    '64/Limeira',
    '36/Marg.Tietê-Pte Remédios',
    '76/Marília',
    '22/Mauá',
    '3/Mooca',
    '6/N.Senhora do Ó',
    '17/Osasco',
    '1/Parque D.Pedro II',
    '44/Paulínia',
    '48/Paulínia-Sta Terezinha',
    '66/Perus',
    '39/Pico do Jaraguá',
    '27/Pinheiros',
    '77/Piracicaba',
    '78/Presidente Prudente',
    '84/Ribeirão Preto',
    '59/Rio Claro-Jd.Guanabara',
    '18/S.André-Capuava',
    '38/S.Bernardo-Centro',
    '19/S.Bernardo-Paulicéia',
    '55/S.José Campos',
    '56/S.José Campos-Jd.Satelite',
    '57/S.José Campos-Vista Verde',
    '53/Santa Gertrudes',
    '2/Santana',
    '16/Santo Amaro',
    '82/Santos',
    '83/Santos-Ponta da Praia',
    '7/São Caetano do Sul',
    '80/São José do Rio Preto',
    '51/Sorocaba',
    '20/Taboão da Serra',
    '94/Tatuí',
    '58/Taubaté'
]

In [7]:
# begin_date_str = '2013-05-08'
# end_date_str = '2020-07-07'

begin_date_str = '2020-07-08'
end_date_str = '2020-09-10'

In [8]:
begin_date = datetime.datetime.strptime(begin_date_str, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d').date()

dates = pd.date_range(begin_date, end_date).tolist()

In [9]:
air_quality = pd.DataFrame(columns=[
        'Date', 
        'Hora', 
        'Station', 
        'BENZENO ug/m3 Média Horária', 
        'CO ppm Índice / Qualidade',
        'MP10 µg/m³ Índice / Qualidade',
        'MP2.5 µg/m³ Índice / Qualidade', 
        'NO2 µg/m³ Índice / Qualidade',
        'O3 µg/m³ Índice / Qualidade', 
        'SO2 µg/m³ Índice / Qualidade',
        'TOLUENO ug/m3 Média Horária', 
        'TRS ppb Média Horária'])

# Get data for all dates and stations
for date in dates:
    for station in STATIONS:
        data = get_data(date.date(), station)
        air_quality = air_quality.append(data, ignore_index=True)

Americana 08/07/2020
Araçatuba 08/07/2020
Araraquara 08/07/2020
Bauru 08/07/2020
Campinas-Centro 08/07/2020
Campinas-Taquaral 08/07/2020
Campinas-V.União 08/07/2020
Capão Redondo 08/07/2020
Carapicuíba 08/07/2020
Catanduva 08/07/2020
Cerqueira César 08/07/2020
Cid.Universitária-USP-Ipen 08/07/2020
Congonhas 08/07/2020
Cubatão-Centro 08/07/2020
Cubatão-V.Parisi 08/07/2020
Cubatão-Vale do Mogi 08/07/2020
Diadema 08/07/2020
Grajaú-Parelheiros 08/07/2020
Guaratinguetá 08/07/2020
Guarulhos-Paço Municipal 08/07/2020
Guarulhos-Pimentas 08/07/2020
Ibirapuera 08/07/2020
Interlagos 08/07/2020
Itaim Paulista 08/07/2020
Itaquera 08/07/2020
Jacareí 08/07/2020
Jaú 08/07/2020
Jundiaí 08/07/2020
Limeira 08/07/2020
Marg.Tietê-Pte Remédios 08/07/2020
Marília 08/07/2020
Mauá 08/07/2020
Mooca 08/07/2020
N.Senhora do Ó 08/07/2020
Osasco 08/07/2020
Parque D.Pedro II 08/07/2020
Paulínia 08/07/2020
Paulínia-Sta Terezinha 08/07/2020
Perus 08/07/2020
Pico do Jaraguá 08/07/2020
Pinheiros 08/07/2020
Piracicaba 08

Guaratinguetá 13/07/2020
Guarulhos-Paço Municipal 13/07/2020
Guarulhos-Pimentas 13/07/2020
Ibirapuera 13/07/2020
Interlagos 13/07/2020
Itaim Paulista 13/07/2020
Itaquera 13/07/2020
Jacareí 13/07/2020
Jaú 13/07/2020
Jundiaí 13/07/2020
Limeira 13/07/2020
Marg.Tietê-Pte Remédios 13/07/2020
Marília 13/07/2020
Mauá 13/07/2020
Mooca 13/07/2020
N.Senhora do Ó 13/07/2020
Osasco 13/07/2020
Parque D.Pedro II 13/07/2020
Paulínia 13/07/2020
Paulínia-Sta Terezinha 13/07/2020
Perus 13/07/2020
Pico do Jaraguá 13/07/2020
Pinheiros 13/07/2020
Piracicaba 13/07/2020
Presidente Prudente 13/07/2020
Ribeirão Preto 13/07/2020
Rio Claro-Jd.Guanabara 13/07/2020
S.André-Capuava 13/07/2020
S.Bernardo-Centro 13/07/2020
S.Bernardo-Paulicéia 13/07/2020
S.José Campos 13/07/2020
S.José Campos-Jd.Satelite 13/07/2020
S.José Campos-Vista Verde 13/07/2020
Santa Gertrudes 13/07/2020
Santana 13/07/2020
Santo Amaro 13/07/2020
Santos 13/07/2020
Santos-Ponta da Praia 13/07/2020
São Caetano do Sul 13/07/2020
São José do Rio Pr

Paulínia-Sta Terezinha 18/07/2020
Perus 18/07/2020
Pico do Jaraguá 18/07/2020
Pinheiros 18/07/2020
Piracicaba 18/07/2020
Presidente Prudente 18/07/2020
Ribeirão Preto 18/07/2020
Rio Claro-Jd.Guanabara 18/07/2020
S.André-Capuava 18/07/2020
S.Bernardo-Centro 18/07/2020
S.Bernardo-Paulicéia 18/07/2020
S.José Campos 18/07/2020
S.José Campos-Jd.Satelite 18/07/2020
S.José Campos-Vista Verde 18/07/2020
Santa Gertrudes 18/07/2020
Santana 18/07/2020
Santo Amaro 18/07/2020
Santos 18/07/2020
Santos-Ponta da Praia 18/07/2020
São Caetano do Sul 18/07/2020
São José do Rio Preto 18/07/2020
Sorocaba 18/07/2020
Taboão da Serra 18/07/2020
Tatuí 18/07/2020
Taubaté 18/07/2020
Americana 19/07/2020
Araçatuba 19/07/2020
Araraquara 19/07/2020
Bauru 19/07/2020
Campinas-Centro 19/07/2020
Campinas-Taquaral 19/07/2020
Campinas-V.União 19/07/2020
Capão Redondo 19/07/2020
Carapicuíba 19/07/2020
Catanduva 19/07/2020
Cerqueira César 19/07/2020
Cid.Universitária-USP-Ipen 19/07/2020
Congonhas 19/07/2020
Cubatão-Centro 

Santo Amaro 23/07/2020
Santos 23/07/2020
Santos-Ponta da Praia 23/07/2020
São Caetano do Sul 23/07/2020
São José do Rio Preto 23/07/2020
Sorocaba 23/07/2020
Taboão da Serra 23/07/2020
Tatuí 23/07/2020
Taubaté 23/07/2020
Americana 24/07/2020
Araçatuba 24/07/2020
Araraquara 24/07/2020
Bauru 24/07/2020
Campinas-Centro 24/07/2020
Campinas-Taquaral 24/07/2020
Campinas-V.União 24/07/2020
Capão Redondo 24/07/2020
Carapicuíba 24/07/2020
Catanduva 24/07/2020
Cerqueira César 24/07/2020
Cid.Universitária-USP-Ipen 24/07/2020
Congonhas 24/07/2020
Cubatão-Centro 24/07/2020
Cubatão-V.Parisi 24/07/2020
Cubatão-Vale do Mogi 24/07/2020
Diadema 24/07/2020
Grajaú-Parelheiros 24/07/2020
Guaratinguetá 24/07/2020
Guarulhos-Paço Municipal 24/07/2020
Guarulhos-Pimentas 24/07/2020
Ibirapuera 24/07/2020
Interlagos 24/07/2020
Itaim Paulista 24/07/2020
Itaquera 24/07/2020
Jacareí 24/07/2020
Jaú 24/07/2020
Jundiaí 24/07/2020
Limeira 24/07/2020
Marg.Tietê-Pte Remédios 24/07/2020
Marília 24/07/2020
Mauá 24/07/2020
Mo

Catanduva 29/07/2020
Cerqueira César 29/07/2020
Cid.Universitária-USP-Ipen 29/07/2020
Congonhas 29/07/2020
Cubatão-Centro 29/07/2020
Cubatão-V.Parisi 29/07/2020
Cubatão-Vale do Mogi 29/07/2020
Diadema 29/07/2020
Grajaú-Parelheiros 29/07/2020
Guaratinguetá 29/07/2020
Guarulhos-Paço Municipal 29/07/2020
Guarulhos-Pimentas 29/07/2020
Ibirapuera 29/07/2020
Interlagos 29/07/2020
Itaim Paulista 29/07/2020
Itaquera 29/07/2020
Jacareí 29/07/2020
Jaú 29/07/2020
Jundiaí 29/07/2020
Limeira 29/07/2020
Marg.Tietê-Pte Remédios 29/07/2020
Marília 29/07/2020
Mauá 29/07/2020
Mooca 29/07/2020
N.Senhora do Ó 29/07/2020
Osasco 29/07/2020
Parque D.Pedro II 29/07/2020
Paulínia 29/07/2020
Paulínia-Sta Terezinha 29/07/2020
Perus 29/07/2020
Pico do Jaraguá 29/07/2020
Pinheiros 29/07/2020
Piracicaba 29/07/2020
Presidente Prudente 29/07/2020
Ribeirão Preto 29/07/2020
Rio Claro-Jd.Guanabara 29/07/2020
S.André-Capuava 29/07/2020
S.Bernardo-Centro 29/07/2020
S.Bernardo-Paulicéia 29/07/2020
S.José Campos 29/07/2020


...

Grajaú-Parelheiros 19/08/2020
Guaratinguetá 19/08/2020
Guarulhos-Paço Municipal 19/08/2020
Guarulhos-Pimentas 19/08/2020
Ibirapuera 19/08/2020
Interlagos 19/08/2020
Itaim Paulista 19/08/2020
Itaquera 19/08/2020
Jacareí 19/08/2020
Jaú 19/08/2020
Jundiaí 19/08/2020
Limeira 19/08/2020
Marg.Tietê-Pte Remédios 19/08/2020
Marília 19/08/2020
Mauá 19/08/2020
Mooca 19/08/2020
N.Senhora do Ó 19/08/2020
Osasco 19/08/2020
Parque D.Pedro II 19/08/2020
Paulínia 19/08/2020
Paulínia-Sta Terezinha 19/08/2020
Perus 19/08/2020
Pico do Jaraguá 19/08/2020
Pinheiros 19/08/2020
Piracicaba 19/08/2020
Presidente Prudente 19/08/2020
Ribeirão Preto 19/08/2020
Rio Claro-Jd.Guanabara 19/08/2020
S.André-Capuava 19/08/2020
S.Bernardo-Centro 19/08/2020
S.Bernardo-Paulicéia 19/08/2020
S.José Campos 19/08/2020
S.José Campos-Jd.Satelite 19/08/2020
S.José Campos-Vista Verde 19/08/2020
Santa Gertrudes 19/08/2020
Santana 19/08/2020
Santo Amaro 19/08/2020
Santos 19/08/2020
Santos-Ponta da Praia 19/08/2020
São Caetano do Sul

Paulínia 24/08/2020
Paulínia-Sta Terezinha 24/08/2020
Perus 24/08/2020
Pico do Jaraguá 24/08/2020
Pinheiros 24/08/2020
Piracicaba 24/08/2020
Presidente Prudente 24/08/2020
Ribeirão Preto 24/08/2020
Rio Claro-Jd.Guanabara 24/08/2020
S.André-Capuava 24/08/2020
S.Bernardo-Centro 24/08/2020
S.Bernardo-Paulicéia 24/08/2020
S.José Campos 24/08/2020
S.José Campos-Jd.Satelite 24/08/2020
S.José Campos-Vista Verde 24/08/2020
Santa Gertrudes 24/08/2020
Santana 24/08/2020
Santo Amaro 24/08/2020
Santos 24/08/2020
Santos-Ponta da Praia 24/08/2020
São Caetano do Sul 24/08/2020
São José do Rio Preto 24/08/2020
Sorocaba 24/08/2020
Taboão da Serra 24/08/2020
Tatuí 24/08/2020
Taubaté 24/08/2020
Americana 25/08/2020
Araçatuba 25/08/2020
Araraquara 25/08/2020
Bauru 25/08/2020
Campinas-Centro 25/08/2020
Campinas-Taquaral 25/08/2020
Campinas-V.União 25/08/2020
Capão Redondo 25/08/2020
Carapicuíba 25/08/2020
Catanduva 25/08/2020
Cerqueira César 25/08/2020
Cid.Universitária-USP-Ipen 25/08/2020
Congonhas 25/08/

Santana 29/08/2020
Santo Amaro 29/08/2020
Santos 29/08/2020
Santos-Ponta da Praia 29/08/2020
São Caetano do Sul 29/08/2020
São José do Rio Preto 29/08/2020
Sorocaba 29/08/2020
Taboão da Serra 29/08/2020
Tatuí 29/08/2020
Taubaté 29/08/2020
Americana 30/08/2020
Araçatuba 30/08/2020
Araraquara 30/08/2020
Bauru 30/08/2020
Campinas-Centro 30/08/2020
Campinas-Taquaral 30/08/2020
Campinas-V.União 30/08/2020
Capão Redondo 30/08/2020
Carapicuíba 30/08/2020
Catanduva 30/08/2020
Cerqueira César 30/08/2020
Cid.Universitária-USP-Ipen 30/08/2020
Congonhas 30/08/2020
Cubatão-Centro 30/08/2020
Cubatão-V.Parisi 30/08/2020
Cubatão-Vale do Mogi 30/08/2020
Diadema 30/08/2020
Grajaú-Parelheiros 30/08/2020
Guaratinguetá 30/08/2020
Guarulhos-Paço Municipal 30/08/2020
Guarulhos-Pimentas 30/08/2020
Ibirapuera 30/08/2020
Interlagos 30/08/2020
Itaim Paulista 30/08/2020
Itaquera 30/08/2020
Jacareí 30/08/2020
Jaú 30/08/2020
Jundiaí 30/08/2020
Limeira 30/08/2020
Marg.Tietê-Pte Remédios 30/08/2020
Marília 30/08/2020

Carapicuíba 04/09/2020
Catanduva 04/09/2020
Cerqueira César 04/09/2020
Cid.Universitária-USP-Ipen 04/09/2020
Congonhas 04/09/2020
Cubatão-Centro 04/09/2020
Cubatão-V.Parisi 04/09/2020
Cubatão-Vale do Mogi 04/09/2020
Diadema 04/09/2020
Grajaú-Parelheiros 04/09/2020
Guaratinguetá 04/09/2020
Guarulhos-Paço Municipal 04/09/2020
Guarulhos-Pimentas 04/09/2020
Ibirapuera 04/09/2020
Interlagos 04/09/2020
Itaim Paulista 04/09/2020
Itaquera 04/09/2020
Jacareí 04/09/2020
Jaú 04/09/2020
Jundiaí 04/09/2020
Limeira 04/09/2020
Marg.Tietê-Pte Remédios 04/09/2020
Marília 04/09/2020
Mauá 04/09/2020
Mooca 04/09/2020
N.Senhora do Ó 04/09/2020
Osasco 04/09/2020
Parque D.Pedro II 04/09/2020
Paulínia 04/09/2020
Paulínia-Sta Terezinha 04/09/2020
Perus 04/09/2020
Pico do Jaraguá 04/09/2020
Pinheiros 04/09/2020
Piracicaba 04/09/2020
Presidente Prudente 04/09/2020
Ribeirão Preto 04/09/2020
Rio Claro-Jd.Guanabara 04/09/2020
S.André-Capuava 04/09/2020
S.Bernardo-Centro 04/09/2020
S.Bernardo-Paulicéia 04/09/2020
S.

Jacareí 09/09/2020
Jaú 09/09/2020
Jundiaí 09/09/2020
Limeira 09/09/2020
Marg.Tietê-Pte Remédios 09/09/2020
Marília 09/09/2020
Mauá 09/09/2020
Mooca 09/09/2020
N.Senhora do Ó 09/09/2020
Osasco 09/09/2020
Parque D.Pedro II 09/09/2020
Paulínia 09/09/2020
Paulínia-Sta Terezinha 09/09/2020
Perus 09/09/2020
Pico do Jaraguá 09/09/2020
Pinheiros 09/09/2020
Piracicaba 09/09/2020
Presidente Prudente 09/09/2020
Ribeirão Preto 09/09/2020
Rio Claro-Jd.Guanabara 09/09/2020
S.André-Capuava 09/09/2020
S.Bernardo-Centro 09/09/2020
S.Bernardo-Paulicéia 09/09/2020
S.José Campos 09/09/2020
S.José Campos-Jd.Satelite 09/09/2020
S.José Campos-Vista Verde 09/09/2020
Santa Gertrudes 09/09/2020
Santana 09/09/2020
Santo Amaro 09/09/2020
Santos 09/09/2020
Santos-Ponta da Praia 09/09/2020
São Caetano do Sul 09/09/2020
São José do Rio Preto 09/09/2020
Sorocaba 09/09/2020
Taboão da Serra 09/09/2020
Tatuí 09/09/2020
Taubaté 09/09/2020
Americana 10/09/2020
Araçatuba 10/09/2020
Araraquara 10/09/2020
Bauru 10/09/2020
Ca

In [10]:
# Rename columns and save
air_quality.columns = [
        'Date', 
        'Hour', 
        'Station', 
        'Benzene', 
        'CO',
        'PM10',
        'PM2.5', 
        'NO2',
        'O3', 
        'SO2',
        'Toluene', 
        'TRS']

In [11]:
def convert_float(v):
    if type(v) == str and ',' in v:
        return float(v.replace(',', '.'))
    return v

In [12]:
# Convert date and time to a single field

air_quality.loc[air_quality['Hour'] == '24:00', 'Hour'] = '00:00'

air_quality['DatetimeStr'] = air_quality['Date'] + ' ' + air_quality['Hour']

air_quality['Datetime'] = pd.to_datetime(air_quality['DatetimeStr'], format='%d/%m/%Y %H:%M')

In [13]:
final = air_quality[['Datetime', 'Station']]
final[['Benzene', 'CO', 'PM10', 'PM2.5', 'NO2', 'O3', 'SO2', 'Toluene', 'TRS']] = air_quality[['Benzene', 'CO', 'PM10', 'PM2.5', 'NO2', 'O3', 'SO2', 'Toluene', 'TRS']].applymap(convert_float)

C:\Users\amand\Programs\Anaconda\envs\study\lib\site-packages\pandas\core\frame.py:2960: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
final.to_csv('data/sp_air_quality_.csv', index=False)